In [1]:
%run ./cs_go_import_and_process_data.ipynb

df_csgo
df_matches - no player data
df_teams - one row per match
df_players - one row per player


In [24]:
# sniper analysis

In [2]:
(df_teams
  .groupBy('snipers_on_team')
  .agg(F.count('snipers_on_team'))
  .orderBy('snipers_on_team')
).show()

+---------------+----------------------+
|snipers_on_team|count(snipers_on_team)|
+---------------+----------------------+
|              0|                   172|
|              1|                  5516|
|              2|                  1825|
|              3|                    61|
+---------------+----------------------+



In [9]:
df_matches_by_number_of_snipers = (df_teams
    .groupBy('team_name')
    .pivot('snipers_on_team')
    .agg(F.count('team_name'))
    .na.fill(0)
    .withColumn('m_ttl',
        F.col('0') + F.col('1') + F.col('2') + F.col('3') 
    )
    .select(
        'team_name',
        F.col('0').alias('m_0'),
        F.col('1').alias('m_1'),
        F.col('2').alias('m_2'),
        F.col('3').alias('m_3'),
        'm_ttl',
    )
    .orderBy(F.col('m_ttl').desc())
)

# df_matches_by_number_of_snipers.show(50)

In [7]:
df_wins_by_number_of_snipers = (df_teams
    .groupBy('team_name')
    .pivot('snipers_on_team')
    .agg(F.sum(F.when(F.col('won') == True, F.lit(1)).otherwise(F.lit(0))).alias('wins'))
    .na.fill(0)
    .withColumn('w_ttl',
        F.col('0') + F.col('1') + F.col('2') + F.col('3') 
    )
    .select(
        'team_name',
        F.col('0').alias('w_0'),
        F.col('1').alias('w_1'),
        F.col('2').alias('w_2'),
        F.col('3').alias('w_3'),
        'w_ttl',
    )
    .orderBy(F.col('w_ttl').desc())
)

# df_wins_by_number_of_snipers.show(50)

In [12]:
df_matches_and_wins_by_number_of_snipers = (df_matches_by_number_of_snipers
    .join(df_wins_by_number_of_snipers, 'team_name', 'left')
    .select(
        '*',
        F.lit(F.round(F.col('w_0') / F.col('m_0'), 2)).alias('p_0'),
        F.lit(F.round(F.col('w_1') / F.col('m_1'), 2)).alias('p_1'),
        F.lit(F.round(F.col('w_2') / F.col('m_2'), 2)).alias('p_2'),
        F.lit(F.round(F.col('w_3') / F.col('m_3'), 2)).alias('p_3'),
    )
    .na.fill(0)
    .withColumn('p_ttl', F.round(F.col('w_ttl') / F.col('m_ttl'), 2))
    .orderBy(F.col('p_ttl').desc())
)

df_matches_and_wins_by_number_of_snipers.show(80)

+--------------------+---+---+---+---+-----+---+---+---+---+-----+----+----+----+----+-----+
|           team_name|m_0|m_1|m_2|m_3|m_ttl|w_0|w_1|w_2|w_3|w_ttl| p_0| p_1| p_2| p_3|p_ttl|
+--------------------+---+---+---+---+-----+---+---+---+---+-----+----+----+----+----+-----+
|                   X|  0|  0|  1|  0|    1|  0|  0|  1|  0|    1| 0.0| 0.0| 1.0| 0.0|  1.0|
|             UKRAINE|  0|  2|  0|  0|    2|  0|  2|  0|  0|    2| 0.0| 1.0| 0.0| 0.0|  1.0|
|          LUMINOSITY|  0| 33| 11|  0|   44|  0| 25| 10|  0|   35| 0.0|0.76|0.91| 0.0|  0.8|
|            ASTRALIS| 13|440| 73|  0|  526|  6|307| 39|  0|  352|0.46| 0.7|0.53| 0.0| 0.67|
|            VALIANCE|  0| 13|  0|  0|   13|  0|  8|  0|  0|    8| 0.0|0.62| 0.0| 0.0| 0.62|
|            VITALITY|  0|117| 13|  0|  130|  0| 68| 11|  0|   79| 0.0|0.58|0.85| 0.0| 0.61|
|               FURIA|  0| 30| 52|  0|   82|  0| 17| 32|  0|   49| 0.0|0.57|0.62| 0.0|  0.6|
|                  SK|  0|206| 34|  0|  240|  0|120| 20|  0|  140| 0.0